In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import pandas as pd

from src.custom_dataset import CustomDataset
from src.handler import Handler

c:\Users\Cai Selvas Sala\GIA_UPC\Personal\DatathonFME\Datathon 2024\handle\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
images_data_path = './data/archive/images/images'
x_train_file_path = './data/x_train.csv'
y_train_file_path = './data/y_train.csv'

In [3]:
batch_size = 32
num_epochs = 2
checkpoint_interval = 1000

In [4]:
dataset = CustomDataset(x_train_path=x_train_file_path, y_train_path=y_train_file_path, image_folder_path=images_data_path)
data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

In [5]:
model = Handler(num_categories_list=dataset.get_num_categories_list())
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=0.001)

In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Training loop
for epoch in range(num_epochs):
	model.train()
	running_loss = 0.0
	correct_predictions = 0
	total_samples = 0
	
	for batch_idx, (images, tabular_data, labels) in enumerate(data_loader):
		# Move data to GPU if available
		images, tabular_data = images.to(device), tabular_data.to(device)
		labels = [label.to(device) for label in labels]  # Move each label tensor to device
		
		# Zero the gradients
		optimizer.zero_grad()
		
		# Forward pass
		outputs = model(images, tabular_data)
		
		print(outputs.shape)
		
		# Compute the loss for each output separately
		loss = sum(criterion(output, label) for output, label in zip(outputs, labels))
		
		# Backward pass and optimize
		loss.backward()
		optimizer.step()
		
		# Update running loss
		running_loss += loss.item()
		
		# Calculate accuracy for each output independently
		for output, label in zip(outputs, labels):
			predicted = torch.argmax(output, dim=1)
			correct_labels = torch.argmax(label, dim=1)
			correct_predictions += (predicted == correct_labels).sum().item()
			total_samples += label.size(0)
		
		# Save model weights periodically
		if (total_samples + 1) % checkpoint_interval == 0:
			checkpoint_path = f'model_checkpoint_{total_samples}.pth'
			torch.save(model.state_dict(), checkpoint_path)
			print(f"Checkpoint saved at '{checkpoint_path}' for {total_samples} samples.")
		
		# Print training stats for the current batch
		if batch_idx % 10 == 0:
			print(f"Epoch [{epoch+1}/{num_epochs}], Batch [{batch_idx}/{len(data_loader)}], "
				  f"Loss: {loss.item():.4f}, Batch Accuracy: {100 * correct_predictions / total_samples:.2f}%")
	
	# Epoch-level loss and accuracy
	epoch_loss = running_loss / len(data_loader)
	epoch_accuracy = 100 * correct_predictions / total_samples
	print(f"Epoch {epoch+1}/{num_epochs} completed: Loss: {epoch_loss:.4f}, Accuracy: {epoch_accuracy:.2f}%")
	
torch.save(model.state_dict(), './models/trained_model.pth')
print("Model saved as './models/trained_model.pth'")


c:\Users\Cai Selvas Sala\GIA_UPC\Personal\DatathonFME\Datathon 2024\handle\src\custom_dataset.py:55: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  tabular_data = torch.tensor(self.tabular_data[idx], dtype=torch.long)
It looks like you are trying to rescale already rescaled images. If the input images have pixel values between 0 and 1, set `do_rescale=False` to avoid rescaling them again.


ValueError: Expected input batch_size (32) to match target batch_size (138).